In [1]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import os 
import pandas as pd
import time
import random
import plotly.express as px
print("libs included")

libs included


In [2]:
class dataset():
    def __init__(self,seq="00"):
        
        self.sequences_dir = 'sequences/{}/'.format(seq)
        self.poses_dir = 'poses/{}.txt'.format(seq)
        
        self.left_img=(cv2.imread(self.sequences_dir+"image_0/"+i,0) for i in sorted(os.listdir(self.sequences_dir+"image_0")) if i.endswith("png"))
        self.right_img=(cv2.imread(self.sequences_dir+"image_1/"+i,0) for i in sorted(os.listdir(self.sequences_dir+"image_1")) if i.endswith("png"))
        self.poses=pd.read_csv(self.poses_dir, delimiter=' ', header=None)
        self.calib=pd.read_csv(self.sequences_dir+"calib.txt", delimiter=' ', header=None, index_col=0)
        
        self.p0=np.array(self.calib.iloc[0]).reshape((3,4))
        self.p1=np.array(self.calib.iloc[1]).reshape((3,4))
        self.Tr = np.array(self.calib.iloc[-1]).reshape((3,4))


    def next_imgs(self):
        return next(self.left_img),next(self.right_img)
    def reset_images(self):
        self.left_img=(cv2.imread(self.sequences_dir+"image_0/"+i,0) for i in sorted(os.listdir(self.sequences_dir+"image_0")) if i.endswith("png"))
        self.right_img=(cv2.imread(self.sequences_dir+"image_1/"+i,0) for i in sorted(os.listdir(self.sequences_dir+"image_1")) if i.endswith("png"))
    
data=dataset()

In [ ]:
#%% Visualization left camera's frames
for i in range(len(os.listdir(data.sequences_dir+"image_0"))):
    cv2.imshow("winname", data.next_imgs()[0])
    key=cv2.waitKey(27)
    if key==ord('q'):
        data.reset_images()
        break
cv2.destroyAllWindows()

In [3]:
def compute_left_disparity_map(img_left, img_right, matcher='bm', verbose=False):
    
    sad_window = 6
    num_disparities = sad_window * 16
    block_size = 11
    matcher_name = matcher
    
    if matcher_name == 'bm':
        matcher = cv2.StereoBM_create(numDisparities=num_disparities,
                                      blockSize=block_size)
        
    elif matcher_name == 'sgbm':
        matcher = cv2.StereoSGBM_create(numDisparities=num_disparities,
                                        minDisparity=0,
                                        blockSize=block_size,
                                        P1 = 8 * 1 * block_size ** 2,
                                        P2 = 32 * 1 * block_size ** 2,
                                        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
    

        
    start = time.perf_counter()
    disp_left = matcher.compute(img_left, img_right).astype(np.float32)/16
    end = time.perf_counter()
    
    if verbose:
        print(f'Time to compute disparity map using Stereo{matcher_name.upper()}', end-start)
        
    return disp_left

# disp = compute_left_disparity_map(data.next_imgs()[0],
#                                   data.next_imgs()[1],
#                                   matcher='sgbm',
#                                   verbose=True)
# plt.figure(figsize=(11,7))
# plt.imshow(disp);


In [ ]:
#%% Visualization disparity frames
for i in range(len(os.listdir(data.sequences_dir+"image_0"))):
    disp = compute_left_disparity_map(data.next_imgs()[0],
                                      data.next_imgs()[1],
                                      matcher='bm',
                                      verbose=False)
    disp/=disp.max()
    disp*=255.
    cv2.imshow("winname", disp.astype(np.uint8))
    key=cv2.waitKey(27)
    if key==ord('q'):
        data.reset_images()
        break
cv2.destroyAllWindows()

In [4]:
def decompose_projection_matrix(p):
    k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(p)
    t = (t / t[3])[:3]
    return k, r, t

def calc_depth_map(disp_left, k_left, t_left, t_right, rectified=True):
    
    if rectified:
        b = t_right[0] - t_left[0]
    else:
        b = t_left[0] - t_right[0]
        
    f = k_left[0][0]
    
    disp_left[disp_left == 0.0] = 0.1
    disp_left[disp_left == -1.0] = 0.1
    
    depth_map = np.ones(disp_left.shape)
    depth_map = f * b / disp_left
    
    return depth_map

# k_left, r_left, t_left = decompose_projectionwert_wertmatrix(data.p0)
# k_right, r_right, t_right = decompose_projection_matrix(data.p1)

# depth = calc_depth_map(disp, k_left, t_left, t_right)


In [ ]:
for i in range(len(os.listdir(data.sequences_dir+"image_0"))):
    k_left, r_left, t_left = decompose_projection_matrix(data.p0)
    k_right, r_right, t_right = decompose_projection_matrix(data.p1)
    disp = compute_left_disparity_map(data.next_imgs()[0],
                                      data.next_imgs()[1],
                                      matcher='sgbm',
                                      verbose=True)
    depth = calc_depth_map(disp, k_left, t_left, t_right)
    depth/=depth.max()
    depth*=255
    cv2.imshow("winname", disp.astype(np.uint8))
    key=cv2.waitKey(27)
    if key==ord('q'):
        data.reset_images()
        break
cv2.destroyAllWindows()


In [5]:
def extract_features(image, detector='sift',GoodP=False,mask=None):
        
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'surf':
        det = cv2.xfeatures2d.SURF_create()
    elif detector == 'orb':
        det = cv2.ORB_create()
    if GoodP:
        pts = cv2.goodFeaturesToTrack(image, 3000, qualityLevel=0.01, minDistance=7)
        kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=15) for f in pts]
        kp, des = det.compute(image, kps)
        

    else:
        kp, des = det.detectAndCompute(image, mask)
    kp=np.array([(k.pt[0], k.pt[1]) for k in kp])
    return kp, des

In [7]:
def match_features(des1, des2, matching='BF', detector='sift', k=2):
    
    if matching == 'BF':
        if detector == 'sift' or detector == 'surf':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
    

        
    return matches

In [8]:
def filter_matches_distance(matches, dist_threshold=0.5):
    filtered_matches = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_matches.append(m)
            
    return filtered_matches 

In [9]:
def drawMatches(img1, img2,kp1,kp2 ,matches):
    merge_img=cv2.hconcat([img1,img2])
    for m in matches:
        p1 = kp1[m.queryIdx]
        p2 = kp2[m.trainIdx]

        x1,y1=map(lambda x:int(round(x)),p1)
        x2,y2=map(lambda x:int(round(x)),p2)
        cv2.circle(merge_img, (x1,y1), 3, (255))

        cv2.circle(merge_img, (img1.shape[1]+x2,y2), 3,(255))
        cv2.line(merge_img, (x1,y1), (img1.shape[1]+x2,y2), (255))
    return merge_img

In [10]:
for i in range(len(os.listdir(data.sequences_dir+"image_0"))):
    s=time.perf_counter()
    img_l=data.next_imgs()[0]
    if not i:
        last_img_l=img_l
        continue
    keypoints_1, descriptors_1 = extract_features(img_l,detector='sift',mask=None)
    keypoints_2, descriptors_2 = extract_features(last_img_l,mask=None)
    matches=match_features(descriptors_1, descriptors_2, matching='BF', detector='sift', k=2)
    matches=filter_matches_distance(matches,dist_threshold=0.5)
    image_matches = drawMatches(img_l, last_img_l, keypoints_1,keypoints_2,matches)
    cv2.namedWindow("winname",cv2.WINDOW_NORMAL)
    cv2.imshow("winname", image_matches)
    key=cv2.waitKey(27)
    if key==ord('q'):
        data.reset_images()
        break
    last_img_l=img_l
    print(round(1/(time.perf_counter()-s),4), "fps")
cv2.destroyAllWindows()


0.6178 fps
1.2471 fps
1.691 fps


In [10]:
def visualize_matches(image1, kp1, image2, kp2, match):
    image_matches = cv2.drawMatches(image1, kp1, image2, kp2, match, None, flags=2)
    plt.figure(figsize=(16, 6), dpi=100)
    plt.imshow(image_matches)

In [12]:
def estimate_motion(matches, kp1, kp2, k, depth1=None, max_depth=3000):
    
    rmat = np.eye(3)
    tvec = np.zeros((3, 1))
    
    image1_points = np.float32([kp1[m.queryIdx] for m in matches])
    image2_points = np.float32([kp2[m.trainIdx] for m in matches])
    if depth1 is not None :
        cx = k[0, 2]
        cy = k[1, 2]
        fx = k[0, 0]
        fy = k[1, 1]
        
        object_points = np.zeros((0, 3))
        delete = []
        
        for i, (u, v) in enumerate(image1_points):
            z = depth1[int(round(v)), int(round(u))]
            
            if z > max_depth:
                delete.append(i)
                continue
                
            x = z * (u - cx) / fx
            y = z * (v - cy) / fy
            object_points = np.vstack([object_points, np.array([x, y, z])])            
        image1_points = np.delete(image1_points, delete, 0)
        image2_points = np.delete(image2_points, delete, 0)
        
        _, rvec, tvec, inliers = cv2.solvePnPRansac(object_points, image2_points, k, None)
        rmat = cv2.Rodrigues(rvec)[0]
    else:
        # Compute the essential matrix
        essential_matrix, mask = cv2.findEssentialMat(image1_points,image2_points , focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=3.0)
        # Recover the relative pose of the cameras
        _, rmat, tvec, mask = cv2.recoverPose(essential_matrix, image1_points, image2_points)
    return rmat, tvec, image1_points, image2_points
        

In [17]:
def mse(m1,m2):
    m1,m2=m1.flatten(),m2.flatten()
    return (sum((m2-m1)** 2))/m1.shape[0]


In [25]:
data.reset_images()

In [26]:
s=time.perf_counter()

last_img_l,last_img_r=data.next_imgs()

# disp=compute_left_disparity_map(last_img_l,last_img_r)
disp=compute_left_disparity_map(last_img_l,
                                  last_img_r,
                                  matcher='sgbm')

k_left, r_left, t_left = decompose_projection_matrix(data.p0)
k_right, r_right, t_right = decompose_projection_matrix(data.p1)

depth = calc_depth_map(disp, k_left, t_left, t_right)


img_l=data.next_imgs()[0]
keypoints_1, descriptors_1 = extract_features(img_l,detector='orb',GoodP=True,mask=None,)
keypoints_2, descriptors_2 = extract_features(last_img_l,detector='orb',GoodP=True,mask=None)
matches=match_features(descriptors_1, descriptors_2, matching='BF', detector='orb',)
matches=filter_matches_distance(matches,0.3)
image_matches = drawMatches(img_l, last_img_l, keypoints_1,keypoints_2,matches)
# plt.figure(figsize=(16, 6), dpi=100)
# plt.imshow(image_matches)
k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(data.p0)
rmat, tvec, image1_points, image2_points =estimate_motion(matches=matches,kp1=keypoints_1,kp2=keypoints_2,k=k,depth1=depth)
transformation_matrix = np.hstack([rmat, tvec])
print((time.perf_counter()-s),'s')


0.16787234999992506 s


In [27]:
m1=np.array(data.poses.iloc[1]).reshape((3,4)).round(4)

In [28]:
print(mse(m1,transformation_matrix))

0.002988347569563586


sgbm,sift ,gp=false  
süre:1.55
mse: 0.0026911559834271546

sgbm,sift ,gp=True  
mse: 0.0036083649504361514

sgbm,orb ,gp=True  
süre:0.29
mse: 0.002988347569563586

sgbm,orb ,gp=False  
süre:0.31
mse: 0.0036566450577993673

bm,orb ,gp=True  
mse: 0.0035251248359337507

bm, orb ,gp=False  
mse: 0.0032920821980853827

bm,sift ,gp=false  
mse: 0.0028242799111296573

bm,sift ,gp=True  
mse: 0.005100526103442996

In [ ]:
def VisualOdometry(matcher='sgbm',detector='orb',GoodP=True,dist_threshol=None):
    pass